In [16]:
# implement https://arxiv.org/pdf/1705.08690.pdf on avalanche framework on permuted MNIST
# https://aahaanmaini.medium.com/mimicking-human-continual-learning-in-a-neural-network-c15e1ae11d70
#continual learning
from avalanche.benchmarks.classic import PermutedMNIST, SplitMNIST
from avalanche.models import SimpleMLP


# CL Benchmark Creation
test_BM = PermutedMNIST(n_experiences=5, seed=12345)
train_BM = SplitMNIST(n_experiences=5)

In [12]:
from torch import nn
from torch.nn import Module

#Naive approach
model_naive = SimpleMLP(num_classes=10, hidden_size = 400, hidden_layers=2)

#Continual Learning approach
model_cl_scholar = SimpleMLP(num_classes=10, hidden_size = 400, hidden_layers=2)

#generative_adversarial_network for continual learning

class Generator(Module):
    def __init__(self):
        super(Generator, self).__init__()
        self.main = module.Sequential(
            Module.Linear(100, 256),
            Module.LeakyReLU(0.2, inplace=True),
            Module.Linear(256, 256),
            Module.BatchNorm1d(256),
            Module.LeakyReLU(0.2, inplace=True),
            Module.Linear(256, 784),
            Module.Tanh()
        )

    def forward(self, input):
        return self.main(input)


In [13]:
from torch.optim import SGD
from torch.nn import CrossEntropyLoss
from avalanche.training.supervised import Naive
from avalanche.training.plugins import EvaluationPlugin
from avalanche.evaluation.metrics import accuracy_metrics

optimizer = SGD(model_naive.parameters(), lr=0.001, momentum=0.9)
criterion = CrossEntropyLoss()


# Continual learning strategy
cl_strategy_naive = Naive(
    model_naive, optimizer, criterion, train_mb_size=32, train_epochs=2, 
    eval_mb_size=32)

In [17]:
#train naive model on train stream
#train GAN on 50% train stream data and 50% generated data
#train CL on 50% GAN generated data and 50% train stream data

results = []
# iterating over the train stream
for experience in train_BM.train_stream:
    print("Start of task ", experience.task_label)
    print('Classes in this task:', experience.classes_in_this_experience)

    # The current Pytorch training set can be easily recovered through the 
    # experience
    current_training_set = experience.dataset
    current_test_set = test_BM.test_stream[experience.current_experience].dataset
    
    #train naive model on train stream
    cl_strategy_naive.train(current_training_set)

    #test naive model on test stream
    results.append(cl_strategy_naive.eval(current_test_set)


Start of task  0
Classes in this task: [4, 6]
Task 0
This task contains 11760 training examples
This task contains 10000 test examples
Start of task  0
Classes in this task: [1, 3]
Task 0
This task contains 12873 training examples
This task contains 10000 test examples
Start of task  0
Classes in this task: [8, 2]
Task 0
This task contains 11809 training examples
This task contains 10000 test examples
Start of task  0
Classes in this task: [5, 7]
Task 0
This task contains 11686 training examples
This task contains 10000 test examples
Start of task  0
Classes in this task: [0, 9]
Task 0
This task contains 11872 training examples
This task contains 10000 test examples


In [ ]:
print("Naive approach results: ", results_naive)